#  Import dependencies


In [1]:
import os 
import time
from imageio import imread
import cv2

In [2]:
from unet_pp import Xnet
from unet_pp.data import *
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


# Config

In [3]:
# set the root path of train data
data_path = r'./data'
# train image path
images_folder = 'train_images'
# train mask path
labels_folder = 'train_labels'

# val image path
valid_images_folder = 'val_images'
# val mask path
valid_labels_folder = 'val_labels'


# model save path
cache_path = data_path + '/cache'

save_model_filename = 'ct_angel'

batch_size = 2
epochs = 2

img_size = 512
train_samples = 4   # change according to actual
valid_samples = 2   # change according to actual
# input_size
input_size = (img_size, img_size, 3)

# Data augmentation

In [4]:
# train data
train_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

trainGene = trainGenerator(batch_size, data_path, images_folder, labels_folder, train_gen_args,
                        image_color_mode="rgb", mask_color_mode="grayscale",
                        target_size=(img_size,img_size))

# valid data
valid_gen_args = dict(fill_mode='nearest')

validGene = trainGenerator(batch_size, data_path, valid_images_folder, valid_labels_folder, valid_gen_args,
                        image_color_mode="rgb", mask_color_mode="grayscale",
                        target_size=(img_size,img_size))

# Train

In [5]:
# model save path
if not os.path.exists(cache_path):
    os.mkdir(cache_path)

In [6]:
# create unet_pp model and set params

model = Xnet(input_shape=input_size, backbone_name='resnet50', encoder_weights='imagenet', decoder_block_type='transpose')

model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

now = time.strftime("%y-%m-%d_%H_%M", time.localtime(time.time()))

model_name = save_model_filename + "_" + now + '_{epoch:02d}-{val_acc:.3f}.hdf5'
abs_model_name = os.path.join(cache_path, model_name)

model_checkpoint = ModelCheckpoint(abs_model_name, monitor='val_loss', verbose=2, save_best_only=True)
early_stop = EarlyStopping(monitor='val_loss', patience=10)
callbacks = [early_stop, model_checkpoint]

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
# begin train
history = model.fit_generator(trainGene, steps_per_epoch=train_samples // batch_size, epochs=epochs, 
                    validation_data=validGene, validation_steps=valid_samples // batch_size, 
                    callbacks=callbacks)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
Found 2 images belonging to 1 classes.
Found 4 images belonging to 1 classes.
Found 4 images belonging to 1 classes.
Found 2 images belonging to 1 classes.
2/2 [==============================] - 21s 11s/step - loss: 0.6574 - acc: 0.6127 - val_loss: 0.6311 - val_acc: 0.6486

Epoch 00001: val_loss improved from inf to 0.63108, saving model to ./data/cache\ct_angel_20-03-14_10_46_01-0.649.hdf5
Epoch 2/2
2/2 [==============================] - 1s 405ms/step - loss: 0.5981 - acc: 0.7006 - val_loss: 0.5508 - val_acc: 0.7393

Epoch 00002: val_loss improved from 0.63108 to 0.55077, saving model to ./data/cache\ct_angel_20-03-14_10_46_02-0.739.hdf5


# Predict

In [8]:
# load model weights

model_file_name = 'ct_angel_20-03-14_10_38_02-0.770.hdf5'
model_path = os.path.join(cache_path, model_file_name)

model = Xnet(input_shape=input_size, backbone_name='resnet50', encoder_weights='imagenet', decoder_block_type='transpose')

print('load: ', model_path)
model.load_weights(model_path)

load:  ./data/cache\ct_angel_20-03-14_10_38_02-0.770.hdf5


In [9]:
# predict and save image
test_img_path = data_path + r'\test_images'
test_label_path = data_path + r'\test_labels'
result_path = data_path + r'\prd_result'
if not os.path.exists(result_path):
    os.mkdir(result_path)
    
test_files = os.listdir(test_img_path)
predict_area_num = 3

In [10]:
def takeLength(elem):
    return len(elem)

In [11]:
for file in test_files:
    fpath,fname = os.path.split(file)

    out_fname = os.path.join(result_path, fname)

    
    src_img = imread(os.path.join(test_img_path, file))
    img = src_img
    
    img = trans.resize(img,(img_size,img_size))
    
    # img from (512, 512, 3) trans to (1, 512, 512, 3)
    img = np.reshape(img,(1,)+img.shape)


    # predict
    results = model.predict(np.array(img),verbose=1)

    gray_img = results[0][:,:,0]

    # to gray
    bw_heatmap = np.uint8(255 * gray_img)
    bw_heatmap[bw_heatmap <= 127] = 0
    bw_heatmap[bw_heatmap > 127] = 255
    _, ai_ctrs, _ = cv2.findContours(bw_heatmap.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    ai_ctrs.sort(key=takeLength, reverse=True)

    # choose the bigger 
    ai_ctrs = ai_ctrs[:predict_area_num]

    # draw box
    for c in ai_ctrs:        
        # find bounding box coordinates
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(src_img, (x,y), (x+w, y+h), (255, 0, 0), 1)
        

    # save result image
    io.imsave(out_fname, src_img)
    


1/1 [==============================] - 0s 65ms/step
